In [1]:
import fsspec

fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True, use_ssl=False) # For now SSL false is solving my cert issues **shrug**
fs_write = fsspec.filesystem('') # For now we are writing to local disk

In [2]:
dbofs_files = fs_read.glob('s3://nextgen-dmac/nos/nos.dbofs.fields.f*.*.t*z.nc.zarr')

dbofs_files = sorted(['s3://'+f for f in dbofs_files])
dbofs_files

['s3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t00z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t06z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t12z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t18z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t00z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t06z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t12z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t18z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t00z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t06z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t12z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t18z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230318.t00z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230318.t06z.nc.zarr',
 's3://nextgen-dmac/

In [10]:
import re
import datetime
from typing import Tuple

def extract_forecast_time_index(key) -> Tuple[str, int]:
    offset, model_date, model_hour = re.search(r'f(\d{3}).(\d{8}).t(\d{2})', key).groups()
    model_date = datetime.datetime.strptime(f'{model_date}T{model_hour}', '%Y%m%dT%H') + datetime.timedelta(hours=int(offset))
    model_date_key = model_date.strftime('%Y%m%dT%H')
    return model_date_key, int(offset)

In [14]:
extract_forecast_time_index('s3://nextgen-dmac/nos/nos.dbofs.fields.f026.20230321.t12z.nc.zarr')

indexes = {}

for f in dbofs_files:
    model_date_key, offset = extract_forecast_time_index(f)
    if model_date_key not in indexes:
        indexes[model_date_key] = [offset, f]
    else: 
        if offset < indexes[model_date_key][0]:
            indexes[model_date_key] = [offset, f]

indexes 

{'20230315T01': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t00z.nc.zarr'],
 '20230315T07': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t06z.nc.zarr'],
 '20230315T13': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t12z.nc.zarr'],
 '20230315T19': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t18z.nc.zarr'],
 '20230316T01': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t00z.nc.zarr'],
 '20230316T07': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t06z.nc.zarr'],
 '20230316T13': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t12z.nc.zarr'],
 '20230316T19': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t18z.nc.zarr'],
 '20230317T01': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t00z.nc.zarr'],
 '20230317T07': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t06z.nc.zarr'],
 '20230317T13': [1,
  's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t12z

In [15]:
dbofs_best_files = [x[1] for x in list(indexes.values())]
dbofs_best_files

['s3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t00z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t06z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t12z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230315.t18z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t00z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t06z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t12z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230316.t18z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t00z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t06z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t12z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230317.t18z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230318.t00z.nc.zarr',
 's3://nextgen-dmac/nos/nos.dbofs.fields.f001.20230318.t06z.nc.zarr',
 's3://nextgen-dmac/

In [17]:
from kerchunk.combine import MultiZarrToZarr
import ujson

mzz = MultiZarrToZarr(
    dbofs_best_files, 
    remote_protocol='s3', 
    remote_options={'anon': True, 'use_ssl': False},
    concat_dims=['ocean_time'],
    identical_dims=['eta_rho', 'xi_rho', 's_rho', 'eta_psi', 'xi_psi', 's_w', 'eta_u', 'xi_u', 'eta_v', 'xi_v', 'lat_rho', 'lat_psi', 'lat_u', 'lat_v', 'lon_rho', 'lon_psi', 'lon_u', 'lon_v']
)

d = mzz.translate()

with fs_write.open('nos.dbofs.fields.best.zarr', 'wb') as f:
    f.write(ujson.dumps(d).encode())
    print('Successfully wrote kerchunked nos collection to disk')

Successfully wrote kerchunked nos collection to disk


In [5]:
import xarray as xr

# open dataset as zarr object using fsspec reference file system and xarray
fs = fsspec.filesystem("reference", fo='s3://nextgen-dmac/nos/nos.dbofs.fields.best.nc.zarr', remote_protocol='s3', remote_options={'anon':True, 'use_ssl': False}, target_protocol='s3', target_options={'anon':True, 'use_ssl': False})
m = fs.get_mapper("") 

ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), 
                      chunks={'ocean_time':1})
ds

FileNotFoundError: nextgen-dmac/nos/nos.dbofs.fields.best.zarr